In [2]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import random
import warnings
warnings.filterwarnings('ignore')

In [3]:
class Config:
    seed=2024
    image_transform=transforms.Resize((512, 512))
    num_folds=5

In [4]:
models=[]
# the first 5 models are those with cross validation
for i in range(Config.num_folds):
    model = torch.load(f'/kaggle/input/hms-baseline-resnet34d-512-512-training-5-folds/HMS_resnet_fold{i}.pth')
    models.append(model)
# the last model is without cross validation    
model = torch.load("/kaggle/input/hms-baseline-resnet34d-512-512-training/HMS_resnet.pth")
models.append(model)

In [5]:
models[0]

ResNet(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
   

In [6]:
def seed_everything(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
seed_everything(Config.seed)

In [7]:
test_df=pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/test.csv")
submission=pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")
submission=submission.merge(test_df,on='eeg_id',how='left')
submission['path']=submission['spectrogram_id'].apply(lambda x: "/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/"+str(x)+".parquet" )
submission.head()

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,spectrogram_id,patient_id,path
0,3911565283,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,853520,6885,/kaggle/input/hms-harmful-brain-activity-class...


In [8]:
paths=submission['path'].values

test_preds=[]

for path in paths:
    eps=1e-6
    data=pd.read_parquet(path)
    data = data.fillna(-1).values[:,1:].T
    data=data[:,0:300]#(400,300)
    data=np.clip(data,np.exp(-6),np.exp(10))
    data= np.log(data)

    data_mean=data.mean(axis=(0,1))
    data_std=data.std(axis=(0,1))
    data=(data-data_mean)/(data_std+eps)
    data_tensor = torch.unsqueeze(torch.Tensor(data), dim=0)
    data=Config.image_transform(data_tensor)
                 
    test_pred=[]
                 
    for model in models:
        model.eval()
        with torch.no_grad():
            pred=F.softmax(model(data.unsqueeze(0)))[0]
            pred=pred.detach().cpu().numpy()
        test_pred.append(pred)
                 
    test_pred=np.array(test_pred).mean(axis=0)
    test_preds.append(test_pred)
                 
pred_M3 =np.array(test_preds)

In [11]:
submission = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")
labels = ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

# Assign model predictions to respective columns in the submission DataFrame
for i in range(len(labels)):
    submission[f'{labels[i]}_vote'] = pred_M3[:, i]
# Save the updated DataFrame as the final submission file
submission.to_csv("submission.csv", index=None)

In [12]:
submission.head()

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.059191,0.097527,0.001333,0.430365,0.026279,0.385306


In [13]:
# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
submission.iloc[:,-6:].sum(axis=1)

0    1.0
dtype: float32